## Capstone - Group Rate Analysis and Ranking

In [11]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from shapely.geometry import Point
import calendar
import datetime

In [12]:
jan_18 = pd.read_excel('data/Groups360_201801.xls')
jan_18.head()

,SegmentName,Region,Lat,Lng,State,State Formula,Date,SupplyTY,Supply%Chg,DemandTY,...,TransientOccTY,TransientOcc%Chg,TransientADRTY,TransientADR%Chg,TransientRevPARTY,TransientRevPAR%Chg,SampleProps,CensusProps,SampleRooms,CensusRooms
0,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180101,15383.0,1.8,8483.0,...,46.4,24.6,198.32,19.4,92.00,48.8,46,52,15165,15383
1,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180102,15383.0,1.8,8998.0,...,48.9,16.7,175.86,10.9,85.92,29.4,46,52,15165,15383
2,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180103,15383.0,1.8,10010.0,...,51.8,18.1,176.12,11.4,91.24,31.6,46,52,15165,15383
3,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180104,15383.0,1.8,10187.0,...,51.3,43.8,175.49,3.9,89.95,49.4,46,52,15165,15383
4,"Anaheim/Santa Ana, CA Luxury & Upper Upscale C...","Anaheim/Santa Ana, CA",33.834197,-117.985579,CA,CA,20180105,15383.0,1.8,9779.0,...,49.2,47.9,169.88,-0.1,83.51,47.7,46,52,15165,15383


In [13]:
jan_18.columns.values

array(['SegmentName', 'Region', 'Lat', 'Lng', 'State', 'State Formula',
       'Date', 'SupplyTY', 'Supply%Chg', 'DemandTY', 'Demand%Chg',
       'RevenueTY', 'Rev%Chg', 'OccTY', 'Occ%Chg', 'ADRTY', 'ADR%Chg',
       'RevParTY', 'RevPar%Chg', 'GroupDemandTY', 'GroupRevenueTY',
       'GroupRev%Chg', 'GroupOccTY', 'GroupOcc%Chg', 'GroupADRTY',
       'GroupADR%Chg', 'GroupRevPARTY', 'GroupRevPAR%Chg',
       'ContractDemandTY', 'ContractRevenueTY', 'ContractRev%Chg',
       'ContractOccTY', 'ContractOcc%Chg', 'ContractADRTY',
       'ContractADR%Chg', 'ContractRevPARTY', 'ContractRevPAR%Chg',
       'TransientDemandTY', 'TransientRevenueTY', 'TransientRev%Chg',
       'TransientOccTY', 'TransientOcc%Chg', 'TransientADRTY',
       'TransientADR%Chg', 'TransientRevPARTY', 'TransientRevPAR%Chg',
       'SampleProps', 'CensusProps', 'SampleRooms', 'CensusRooms'],
      dtype=object)

In [14]:
print('There are ' + str(jan_18['Region'].nunique()) + ' locations.')

There are 59 locations.


In [15]:
jan_18.dtypes

SegmentName             object
Region                  object
Lat                    float64
Lng                    float64
State                   object
State Formula           object
Date                     int64
SupplyTY               float64
Supply%Chg             float64
DemandTY               float64
Demand%Chg             float64
RevenueTY              float64
Rev%Chg                float64
OccTY                  float64
Occ%Chg                float64
ADRTY                  float64
ADR%Chg                float64
RevParTY               float64
RevPar%Chg             float64
GroupDemandTY          float64
GroupRevenueTY         float64
GroupRev%Chg           float64
GroupOccTY             float64
GroupOcc%Chg           float64
GroupADRTY             float64
GroupADR%Chg           float64
GroupRevPARTY          float64
GroupRevPAR%Chg        float64
ContractDemandTY       float64
ContractRevenueTY      float64
ContractRev%Chg        float64
ContractOccTY          float64
Contract

#### Convert Date column to datetime type

In [16]:
#df['DateTime'] = df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

jan_18['Date'] = jan_18['Date'].apply(lambda x:  pd.to_datetime(str(x), format='%Y%m%d'))

In [17]:
jan_18.Date.dtype

dtype('<M8[ns]')

In [18]:
jan_18.dtypes

SegmentName                    object
Region                         object
Lat                           float64
Lng                           float64
State                          object
State Formula                  object
Date                   datetime64[ns]
SupplyTY                      float64
Supply%Chg                    float64
DemandTY                      float64
Demand%Chg                    float64
RevenueTY                     float64
Rev%Chg                       float64
OccTY                         float64
Occ%Chg                       float64
ADRTY                         float64
ADR%Chg                       float64
RevParTY                      float64
RevPar%Chg                    float64
GroupDemandTY                 float64
GroupRevenueTY                float64
GroupRev%Chg                  float64
GroupOccTY                    float64
GroupOcc%Chg                  float64
GroupADRTY                    float64
GroupADR%Chg                  float64
GroupRevPART

In [19]:
jan_18.shape

(1829, 50)

#### Aggregate Group Revenue TY and Group Demand TY by Month and Region

In [20]:
jan_18_agg1 = jan_18.groupby(['Region', 'Date', 'Lat', 'Lng'])['GroupDemandTY','GroupRevenueTY'].sum().reset_index()
jan_18_agg1.head()

,Region,Date,Lat,Lng,GroupDemandTY,GroupRevenueTY
0,"Anaheim/Santa Ana, CA",2018-01-01,33.834197,-117.985579,772.0,253539.0
1,"Anaheim/Santa Ana, CA",2018-01-02,33.834197,-117.985579,905.0,127444.0
2,"Anaheim/Santa Ana, CA",2018-01-03,33.834197,-117.985579,1435.0,215608.0
3,"Anaheim/Santa Ana, CA",2018-01-04,33.834197,-117.985579,1691.0,255734.0
4,"Anaheim/Santa Ana, CA",2018-01-05,33.834197,-117.985579,1611.0,257831.0


#### Parse Year and Month from Date column

In [21]:
#df['year'] = df['ArrivalDate'].dt.year
#df['month'] = df['ArrivalDate'].dt.month

jan_18_agg1['Year'] = jan_18_agg1['Date'].dt.year
jan_18_agg1.head()

,Region,Date,Lat,Lng,GroupDemandTY,GroupRevenueTY,Year
0,"Anaheim/Santa Ana, CA",2018-01-01,33.834197,-117.985579,772.0,253539.0,2018
1,"Anaheim/Santa Ana, CA",2018-01-02,33.834197,-117.985579,905.0,127444.0,2018
2,"Anaheim/Santa Ana, CA",2018-01-03,33.834197,-117.985579,1435.0,215608.0,2018
3,"Anaheim/Santa Ana, CA",2018-01-04,33.834197,-117.985579,1691.0,255734.0,2018
4,"Anaheim/Santa Ana, CA",2018-01-05,33.834197,-117.985579,1611.0,257831.0,2018


In [22]:
#dt.month_name() provides the full month name

jan_18_agg1['Month'] = jan_18_agg1['Date'].dt.month
jan_18_agg1.head()

,Region,Date,Lat,Lng,GroupDemandTY,GroupRevenueTY,Year,Month
0,"Anaheim/Santa Ana, CA",2018-01-01,33.834197,-117.985579,772.0,253539.0,2018,1
1,"Anaheim/Santa Ana, CA",2018-01-02,33.834197,-117.985579,905.0,127444.0,2018,1
2,"Anaheim/Santa Ana, CA",2018-01-03,33.834197,-117.985579,1435.0,215608.0,2018,1
3,"Anaheim/Santa Ana, CA",2018-01-04,33.834197,-117.985579,1691.0,255734.0,2018,1
4,"Anaheim/Santa Ana, CA",2018-01-05,33.834197,-117.985579,1611.0,257831.0,2018,1


#### Aggregate data by Weekend and Weekday

In [23]:
jan_18_agg1['DayofWeek'] = jan_18_agg1['Date'].dt.day_name()
jan_18_agg1.head()

,Region,Date,Lat,Lng,GroupDemandTY,GroupRevenueTY,Year,Month,DayofWeek
0,"Anaheim/Santa Ana, CA",2018-01-01,33.834197,-117.985579,772.0,253539.0,2018,1,Monday
1,"Anaheim/Santa Ana, CA",2018-01-02,33.834197,-117.985579,905.0,127444.0,2018,1,Tuesday
2,"Anaheim/Santa Ana, CA",2018-01-03,33.834197,-117.985579,1435.0,215608.0,2018,1,Wednesday
3,"Anaheim/Santa Ana, CA",2018-01-04,33.834197,-117.985579,1691.0,255734.0,2018,1,Thursday
4,"Anaheim/Santa Ana, CA",2018-01-05,33.834197,-117.985579,1611.0,257831.0,2018,1,Friday


#### Subset DF for weekends and then weekdays

In [24]:
jan_18_wknd = jan_18_agg1.loc[jan_18_agg1['DayofWeek'].isin(['Friday', 'Saturday'])]
jan_18_wknd.head()

,Region,Date,Lat,Lng,GroupDemandTY,GroupRevenueTY,Year,Month,DayofWeek
4,"Anaheim/Santa Ana, CA",2018-01-05,33.834197,-117.985579,1611.0,257831.0,2018,1,Friday
5,"Anaheim/Santa Ana, CA",2018-01-06,33.834197,-117.985579,2010.0,437568.0,2018,1,Saturday
11,"Anaheim/Santa Ana, CA",2018-01-12,33.834197,-117.985579,2738.0,535039.0,2018,1,Friday
12,"Anaheim/Santa Ana, CA",2018-01-13,33.834197,-117.985579,2928.0,603487.0,2018,1,Saturday
18,"Anaheim/Santa Ana, CA",2018-01-19,33.834197,-117.985579,3938.0,825676.0,2018,1,Friday


#### Aggregate data by Year, Month, Region

In [26]:
jan_18_agg2 = jan_18_agg1.groupby(['Year', 'Month', 'Region', 'Lat', 'Lng'])['GroupDemandTY','GroupRevenueTY'].sum().reset_index()
jan_18_agg2.head()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY
0,2018,1,"Anaheim/Santa Ana, CA",33.834197,-117.985579,123337.0,27156640.0
1,2018,1,"Atlanta, GA",33.767634,-84.560691,266179.0,48225171.0
2,2018,1,"Austin, TX",30.307462,-98.033602,101873.0,20593488.0
3,2018,1,"Baltimore, MD",39.299236,-76.609383,40616.0,5476141.0
4,2018,1,"Birmingham, AL",33.543682,-86.779633,18656.0,2470727.0


In [27]:
jan_18_agg2.tail()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY
54,2018,1,"Seattle, WA",47.608013,-122.335167,83386.0,15519121.0
55,2018,1,"St Louis, MO-IL",38.627003,-90.199402,53512.0,5942624.0
56,2018,1,"Tampa/St Petersburg, FL",27.964157,-82.452606,112364.0,20961529.0
57,2018,1,"Washington, DC-MD-VA",38.893513,-77.154663,250462.0,45797755.0
58,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0


#### Calculate a monthly ADR using aggregated data by year, month and region

In [28]:
jan_18_agg2['MonthADR'] = (jan_18_agg2.GroupRevenueTY / jan_18_agg2.GroupDemandTY).round(2)
jan_18_agg2.head()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR
0,2018,1,"Anaheim/Santa Ana, CA",33.834197,-117.985579,123337.0,27156640.0,220.18
1,2018,1,"Atlanta, GA",33.767634,-84.560691,266179.0,48225171.0,181.18
2,2018,1,"Austin, TX",30.307462,-98.033602,101873.0,20593488.0,202.15
3,2018,1,"Baltimore, MD",39.299236,-76.609383,40616.0,5476141.0,134.83
4,2018,1,"Birmingham, AL",33.543682,-86.779633,18656.0,2470727.0,132.44


#### Rank the values by MonthADR in Descending Order

In [29]:
jan_18_agg2['Rank'] = jan_18_agg2['MonthADR'].rank(ascending = False)
jan_18_agg2.head(15)

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank
0,2018,1,"Anaheim/Santa Ana, CA",33.834197,-117.985579,123337.0,27156640.0,220.18,15.0
1,2018,1,"Atlanta, GA",33.767634,-84.560691,266179.0,48225171.0,181.18,28.0
2,2018,1,"Austin, TX",30.307462,-98.033602,101873.0,20593488.0,202.15,19.0
3,2018,1,"Baltimore, MD",39.299236,-76.609383,40616.0,5476141.0,134.83,45.0
4,2018,1,"Birmingham, AL",33.543682,-86.779633,18656.0,2470727.0,132.44,47.0
5,2018,1,"Boston, MA",42.361145,-71.057083,99398.0,17518952.0,176.25,31.0
6,2018,1,California Central Coast,35.315787,-120.857870,68965.0,12169253.0,176.46,30.0
7,2018,1,"Charleston, SC",32.776566,-79.930923,31605.0,5452018.0,172.50,34.0
8,2018,1,"Charlotte, NC-SC",35.227085,-80.843124,50792.0,8728029.0,171.84,35.0
9,2018,1,"Chattanooga, TN-GA",35.045631,-85.309677,7766.0,925329.0,119.15,52.0


In [30]:
#data.sort_values("Name", inplace = True)
jan_18_agg2 = jan_18_agg2.sort_values('Rank')
jan_18_agg2

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank
50,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,403.85,1.0
13,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,362.98,2.0
19,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,315.15,3.0
39,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,305.40,4.0
32,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,299.84,5.0
58,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,274.10,6.0
21,2018,1,"Fort Lauderdale, FL",26.139412,-80.133591,75711.0,20507722.0,270.87,7.0
37,2018,1,"New York, NY",40.730610,-73.935242,163800.0,41590578.0,253.91,8.0
18,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,249.44,9.0
43,2018,1,"Phoenix, AZ",33.448376,-112.074036,227198.0,54597598.0,240.31,10.0


#### Drop the NaN values 

In [31]:
#data = data[data['sms'].notnull()]
jan_18_agg3 = jan_18_agg2[jan_18_agg2['MonthADR'].notnull()]
jan_18_agg3.tail()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank
9,2018,1,"Chattanooga, TN-GA",35.045631,-85.309677,7766.0,925329.0,119.15,52.0
38,2018,1,"Norfolk/Virginia Beach, VA",36.850769,-76.285873,19217.0,2157129.0,112.25,53.0
55,2018,1,"St Louis, MO-IL",38.627003,-90.199402,53512.0,5942624.0,111.05,54.0
28,2018,1,"Knoxville, TN",35.964668,-83.926453,12926.0,1323684.0,102.40,55.0
34,2018,1,"Myrtle Beach, SC",33.689060,-78.886696,28000.0,2354348.0,84.08,56.0


In [66]:
jan_18_agg3 = jan_18_agg3.sort_values('Rank')
jan_18_agg3.head(10)

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank
0,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,403.85,1.0
1,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,362.98,2.0
2,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,315.15,3.0
3,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,305.40,4.0
4,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,299.84,5.0
5,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,274.10,6.0
6,2018,1,"Fort Lauderdale, FL",26.139412,-80.133591,75711.0,20507722.0,270.87,7.0
7,2018,1,"New York, NY",40.730610,-73.935242,163800.0,41590578.0,253.91,8.0
8,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,249.44,9.0
9,2018,1,"Phoenix, AZ",33.448376,-112.074036,227198.0,54597598.0,240.31,10.0


In [33]:
jan_18_agg3.shape

(56, 9)

#### Rank all locations based on the Average Daily Rate (ADR) in the month of January

In [34]:
jan_18_agg3 = jan_18_agg3.reset_index(drop=True)
jan_18_agg3.head()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank
0,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,403.85,1.0
1,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,362.98,2.0
2,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,315.15,3.0
3,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,305.40,4.0
4,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,299.84,5.0


#### Subset the monthly DF into 4 Tiers based on ADR Rank and calculate a Comp_Set_ADR for each Tier.  

In [35]:
jan_tier1 = jan_18_agg3.iloc[0:14]
jan_tier1

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank
0,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,403.85,1.0
1,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,362.98,2.0
2,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,315.15,3.0
3,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,305.40,4.0
4,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,299.84,5.0
5,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,274.10,6.0
6,2018,1,"Fort Lauderdale, FL",26.139412,-80.133591,75711.0,20507722.0,270.87,7.0
7,2018,1,"New York, NY",40.730610,-73.935242,163800.0,41590578.0,253.91,8.0
8,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,249.44,9.0
9,2018,1,"Phoenix, AZ",33.448376,-112.074036,227198.0,54597598.0,240.31,10.0


In [36]:
jan_tier1['Tier'] = 1
jan_tier1

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier
0,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,403.85,1.0,1
1,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,362.98,2.0,1
2,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,315.15,3.0,1
3,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,305.40,4.0,1
4,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,299.84,5.0,1
5,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,274.10,6.0,1
6,2018,1,"Fort Lauderdale, FL",26.139412,-80.133591,75711.0,20507722.0,270.87,7.0,1
7,2018,1,"New York, NY",40.730610,-73.935242,163800.0,41590578.0,253.91,8.0,1
8,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,249.44,9.0,1
9,2018,1,"Phoenix, AZ",33.448376,-112.074036,227198.0,54597598.0,240.31,10.0,1


In [37]:
#def comp_index(GroupRevenueTY, GroupDemandTY):
jan_tier1['Tier_ADR'] = (jan_tier1['GroupRevenueTY'].sum()) / (jan_tier1['GroupDemandTY'].sum())

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [39]:
jan_tier1['Tier_ADR'] = jan_tier1['Tier_ADR'].round(2)

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


#### The Tier_ADR shows the Average Rate for all locations in Tier 1.  It becomes the point of comparison for  the ADR for each location.

In [40]:
jan_tier1

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR
0,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,403.85,1.0,1,269.33
1,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,362.98,2.0,1,269.33
2,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,315.15,3.0,1,269.33
3,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,305.40,4.0,1,269.33
4,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,299.84,5.0,1,269.33
5,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,274.10,6.0,1,269.33
6,2018,1,"Fort Lauderdale, FL",26.139412,-80.133591,75711.0,20507722.0,270.87,7.0,1,269.33
7,2018,1,"New York, NY",40.730610,-73.935242,163800.0,41590578.0,253.91,8.0,1,269.33
8,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,249.44,9.0,1,269.33
9,2018,1,"Phoenix, AZ",33.448376,-112.074036,227198.0,54597598.0,240.31,10.0,1,269.33


#### The Comp_Set_Index shows how each location performs against other locations in its Tier.  For example,  Ft. Lauderdale has a Comp_Set_Index of 100.57, meaning its ADR is almost equal to the benchmark index for Tier 1.  San Francisco's rate is almost 50% above Tier 1 benchmark ADR.

In [41]:
jan_tier1['Comp_Set_Index'] = ((jan_tier1['MonthADR'] / jan_tier1['Tier_ADR'])*100).round(2)
jan_tier1

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR,Comp_Set_Index
0,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,403.85,1.0,1,269.33,149.95
1,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,362.98,2.0,1,269.33,134.77
2,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,315.15,3.0,1,269.33,117.01
3,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,305.40,4.0,1,269.33,113.39
4,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,299.84,5.0,1,269.33,111.33
5,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,274.10,6.0,1,269.33,101.77
6,2018,1,"Fort Lauderdale, FL",26.139412,-80.133591,75711.0,20507722.0,270.87,7.0,1,269.33,100.57
7,2018,1,"New York, NY",40.730610,-73.935242,163800.0,41590578.0,253.91,8.0,1,269.33,94.27
8,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,249.44,9.0,1,269.33,92.62
9,2018,1,"Phoenix, AZ",33.448376,-112.074036,227198.0,54597598.0,240.31,10.0,1,269.33,89.23


In [42]:
#Save the file to a csv
jan_tier1.to_csv('jan_tier1.csv', index = False)

#### Tier 2 - Perform the same calculations on the second tier of locations based on the MonthADR rank.

In [67]:
jan_tier2 = jan_18_agg3.iloc[14:28]
jan_tier2

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank
14,2018,1,"Anaheim/Santa Ana, CA",33.834197,-117.985579,123337.0,27156640.0,220.18,15.0
15,2018,1,"Detroit, MI",42.331429,-83.045753,39357.0,8500090.0,215.97,16.0
16,2018,1,"Las Vegas, NV",36.114647,-115.172813,465549.0,99886445.0,214.56,17.0
17,2018,1,"San Diego, CA",32.715736,-117.161087,233940.0,49842949.0,213.06,18.0
18,2018,1,"Austin, TX",30.307462,-98.033602,101873.0,20593488.0,202.15,19.0
19,2018,1,"Sarasota/Bradenton, FL",27.341274,-82.528267,11632.0,2339502.0,201.13,20.0
20,2018,1,"Riverside & San Bernardino, CA",33.945996,-117.539607,87117.0,17358922.0,199.26,21.0
21,2018,1,"Nashville, TN",36.174465,-86.767960,116331.0,22228743.0,191.08,22.0
22,2018,1,"New Orleans, LA",29.951065,-90.071533,153787.0,29209759.0,189.94,23.0
23,2018,1,"San Antonio, TX",29.424349,-98.491142,111015.0,20977779.0,188.96,24.0


In [68]:
jan_tier2['Tier'] = 2

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [69]:
jan_tier2['Tier_ADR'] = ((jan_tier2['GroupRevenueTY'].sum()) / (jan_tier2['GroupDemandTY'].sum())).round(2)
jan_tier2

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR
14,2018,1,"Anaheim/Santa Ana, CA",33.834197,-117.985579,123337.0,27156640.0,220.18,15.0,2,198.76
15,2018,1,"Detroit, MI",42.331429,-83.045753,39357.0,8500090.0,215.97,16.0,2,198.76
16,2018,1,"Las Vegas, NV",36.114647,-115.172813,465549.0,99886445.0,214.56,17.0,2,198.76
17,2018,1,"San Diego, CA",32.715736,-117.161087,233940.0,49842949.0,213.06,18.0,2,198.76
18,2018,1,"Austin, TX",30.307462,-98.033602,101873.0,20593488.0,202.15,19.0,2,198.76
19,2018,1,"Sarasota/Bradenton, FL",27.341274,-82.528267,11632.0,2339502.0,201.13,20.0,2,198.76
20,2018,1,"Riverside & San Bernardino, CA",33.945996,-117.539607,87117.0,17358922.0,199.26,21.0,2,198.76
21,2018,1,"Nashville, TN",36.174465,-86.767960,116331.0,22228743.0,191.08,22.0,2,198.76
22,2018,1,"New Orleans, LA",29.951065,-90.071533,153787.0,29209759.0,189.94,23.0,2,198.76
23,2018,1,"San Antonio, TX",29.424349,-98.491142,111015.0,20977779.0,188.96,24.0,2,198.76


In [70]:
jan_tier2['Comp_Set_Index'] = ((jan_tier2['MonthADR'] / jan_tier2['Tier_ADR'])*100).round(2)
jan_tier2

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR,Comp_Set_Index
14,2018,1,"Anaheim/Santa Ana, CA",33.834197,-117.985579,123337.0,27156640.0,220.18,15.0,2,198.76,110.78
15,2018,1,"Detroit, MI",42.331429,-83.045753,39357.0,8500090.0,215.97,16.0,2,198.76,108.66
16,2018,1,"Las Vegas, NV",36.114647,-115.172813,465549.0,99886445.0,214.56,17.0,2,198.76,107.95
17,2018,1,"San Diego, CA",32.715736,-117.161087,233940.0,49842949.0,213.06,18.0,2,198.76,107.19
18,2018,1,"Austin, TX",30.307462,-98.033602,101873.0,20593488.0,202.15,19.0,2,198.76,101.71
19,2018,1,"Sarasota/Bradenton, FL",27.341274,-82.528267,11632.0,2339502.0,201.13,20.0,2,198.76,101.19
20,2018,1,"Riverside & San Bernardino, CA",33.945996,-117.539607,87117.0,17358922.0,199.26,21.0,2,198.76,100.25
21,2018,1,"Nashville, TN",36.174465,-86.767960,116331.0,22228743.0,191.08,22.0,2,198.76,96.14
22,2018,1,"New Orleans, LA",29.951065,-90.071533,153787.0,29209759.0,189.94,23.0,2,198.76,95.56
23,2018,1,"San Antonio, TX",29.424349,-98.491142,111015.0,20977779.0,188.96,24.0,2,198.76,95.07


In [71]:
jan_tier2.to_csv('jan_tier2.csv', index = False)

#### Tier 3

In [72]:
jan_tier3 = jan_18_agg3.iloc[28:42]
jan_tier3

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank
28,2018,1,"Dallas, TX",32.779167,-96.808891,235527.0,42432064.0,180.16,29.0
29,2018,1,California Central Coast,35.315787,-120.857870,68965.0,12169253.0,176.46,30.0
30,2018,1,"Boston, MA",42.361145,-71.057083,99398.0,17518952.0,176.25,31.0
31,2018,1,"Jacksonville, FL",30.332184,-81.655647,51180.0,8995198.0,175.76,32.0
32,2018,1,"Fort Worth/Arlington, TX",32.768799,-97.309341,76927.0,13379091.0,173.92,33.0
33,2018,1,"Charleston, SC",32.776566,-79.930923,31605.0,5452018.0,172.50,34.0
34,2018,1,"Charlotte, NC-SC",35.227085,-80.843124,50792.0,8728029.0,171.84,35.0
35,2018,1,"Denver, CO",39.742043,-104.991531,86296.0,14672061.0,170.02,36.0
36,2018,1,"Houston, TX",29.749907,-95.358421,123509.0,20622279.0,166.97,37.0
37,2018,1,"Philadelphia, PA-NJ",39.952583,-75.165222,71890.0,11579194.0,161.07,38.0


In [73]:
jan_tier3['Tier'] = 3

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
jan_tier3['Tier_ADR'] = ((jan_tier3['GroupRevenueTY'].sum()) / (jan_tier3['GroupDemandTY'].sum())).round(2)
jan_tier3

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR
28,2018,1,"Dallas, TX",32.779167,-96.808891,235527.0,42432064.0,180.16,29.0,3,170.23
29,2018,1,California Central Coast,35.315787,-120.857870,68965.0,12169253.0,176.46,30.0,3,170.23
30,2018,1,"Boston, MA",42.361145,-71.057083,99398.0,17518952.0,176.25,31.0,3,170.23
31,2018,1,"Jacksonville, FL",30.332184,-81.655647,51180.0,8995198.0,175.76,32.0,3,170.23
32,2018,1,"Fort Worth/Arlington, TX",32.768799,-97.309341,76927.0,13379091.0,173.92,33.0,3,170.23
33,2018,1,"Charleston, SC",32.776566,-79.930923,31605.0,5452018.0,172.50,34.0,3,170.23
34,2018,1,"Charlotte, NC-SC",35.227085,-80.843124,50792.0,8728029.0,171.84,35.0,3,170.23
35,2018,1,"Denver, CO",39.742043,-104.991531,86296.0,14672061.0,170.02,36.0,3,170.23
36,2018,1,"Houston, TX",29.749907,-95.358421,123509.0,20622279.0,166.97,37.0,3,170.23
37,2018,1,"Philadelphia, PA-NJ",39.952583,-75.165222,71890.0,11579194.0,161.07,38.0,3,170.23


In [75]:
jan_tier3['Comp_Set_Index'] = ((jan_tier3['MonthADR'] / jan_tier3['Tier_ADR'])*100).round(2)
jan_tier3

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR,Comp_Set_Index
28,2018,1,"Dallas, TX",32.779167,-96.808891,235527.0,42432064.0,180.16,29.0,3,170.23,105.83
29,2018,1,California Central Coast,35.315787,-120.857870,68965.0,12169253.0,176.46,30.0,3,170.23,103.66
30,2018,1,"Boston, MA",42.361145,-71.057083,99398.0,17518952.0,176.25,31.0,3,170.23,103.54
31,2018,1,"Jacksonville, FL",30.332184,-81.655647,51180.0,8995198.0,175.76,32.0,3,170.23,103.25
32,2018,1,"Fort Worth/Arlington, TX",32.768799,-97.309341,76927.0,13379091.0,173.92,33.0,3,170.23,102.17
33,2018,1,"Charleston, SC",32.776566,-79.930923,31605.0,5452018.0,172.50,34.0,3,170.23,101.33
34,2018,1,"Charlotte, NC-SC",35.227085,-80.843124,50792.0,8728029.0,171.84,35.0,3,170.23,100.95
35,2018,1,"Denver, CO",39.742043,-104.991531,86296.0,14672061.0,170.02,36.0,3,170.23,99.88
36,2018,1,"Houston, TX",29.749907,-95.358421,123509.0,20622279.0,166.97,37.0,3,170.23,98.08
37,2018,1,"Philadelphia, PA-NJ",39.952583,-75.165222,71890.0,11579194.0,161.07,38.0,3,170.23,94.62


In [76]:
jan_tier3.to_csv('jan_tier3.csv', index = False)

#### Tier 4

In [78]:
jan_tier4 = jan_18_agg3.iloc[42:56]
jan_tier4

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank
42,2018,1,"Raleigh/Durham/Chapel Hill, NC",35.787743,-78.644257,33215.0,4780671.0,143.93,43.0
43,2018,1,"Chicago, IL",41.881832,-87.623177,261185.0,36053656.0,138.04,44.0
44,2018,1,"Baltimore, MD",39.299236,-76.609383,40616.0,5476141.0,134.83,45.0
45,2018,1,"Pittsburgh, PA",40.431347,-80.050541,26212.0,3498867.0,133.48,46.0
46,2018,1,"Birmingham, AL",33.543682,-86.779633,18656.0,2470727.0,132.44,47.0
47,2018,1,"Kansas City, MO-KS",39.099724,-94.578331,34002.0,4419129.0,129.97,48.0
48,2018,1,"Louisville, KY-IN",38.328732,-85.764771,19734.0,2534983.0,128.46,49.0
49,2018,1,"Cincinnati, OH-KY-IN",39.103119,-84.512016,19825.0,2463585.0,124.27,50.0
50,2018,1,"Cleveland, OH",41.505493,-81.681290,18122.0,2251858.0,124.26,51.0
51,2018,1,"Chattanooga, TN-GA",35.045631,-85.309677,7766.0,925329.0,119.15,52.0


In [79]:
jan_tier4['Tier'] = 4

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
jan_tier4['Tier_ADR'] = ((jan_tier4['GroupRevenueTY'].sum()) / (jan_tier4['GroupDemandTY'].sum())).round(2)
jan_tier4

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR
42,2018,1,"Raleigh/Durham/Chapel Hill, NC",35.787743,-78.644257,33215.0,4780671.0,143.93,43.0,4,129.27
43,2018,1,"Chicago, IL",41.881832,-87.623177,261185.0,36053656.0,138.04,44.0,4,129.27
44,2018,1,"Baltimore, MD",39.299236,-76.609383,40616.0,5476141.0,134.83,45.0,4,129.27
45,2018,1,"Pittsburgh, PA",40.431347,-80.050541,26212.0,3498867.0,133.48,46.0,4,129.27
46,2018,1,"Birmingham, AL",33.543682,-86.779633,18656.0,2470727.0,132.44,47.0,4,129.27
47,2018,1,"Kansas City, MO-KS",39.099724,-94.578331,34002.0,4419129.0,129.97,48.0,4,129.27
48,2018,1,"Louisville, KY-IN",38.328732,-85.764771,19734.0,2534983.0,128.46,49.0,4,129.27
49,2018,1,"Cincinnati, OH-KY-IN",39.103119,-84.512016,19825.0,2463585.0,124.27,50.0,4,129.27
50,2018,1,"Cleveland, OH",41.505493,-81.681290,18122.0,2251858.0,124.26,51.0,4,129.27
51,2018,1,"Chattanooga, TN-GA",35.045631,-85.309677,7766.0,925329.0,119.15,52.0,4,129.27


In [81]:
jan_tier4['Comp_Set_Index'] = ((jan_tier4['MonthADR'] / jan_tier4['Tier_ADR'])*100).round(2)
jan_tier4

C:\Users\boyer\Anaconda3\envs\geospatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR,Comp_Set_Index
42,2018,1,"Raleigh/Durham/Chapel Hill, NC",35.787743,-78.644257,33215.0,4780671.0,143.93,43.0,4,129.27,111.34
43,2018,1,"Chicago, IL",41.881832,-87.623177,261185.0,36053656.0,138.04,44.0,4,129.27,106.78
44,2018,1,"Baltimore, MD",39.299236,-76.609383,40616.0,5476141.0,134.83,45.0,4,129.27,104.30
45,2018,1,"Pittsburgh, PA",40.431347,-80.050541,26212.0,3498867.0,133.48,46.0,4,129.27,103.26
46,2018,1,"Birmingham, AL",33.543682,-86.779633,18656.0,2470727.0,132.44,47.0,4,129.27,102.45
47,2018,1,"Kansas City, MO-KS",39.099724,-94.578331,34002.0,4419129.0,129.97,48.0,4,129.27,100.54
48,2018,1,"Louisville, KY-IN",38.328732,-85.764771,19734.0,2534983.0,128.46,49.0,4,129.27,99.37
49,2018,1,"Cincinnati, OH-KY-IN",39.103119,-84.512016,19825.0,2463585.0,124.27,50.0,4,129.27,96.13
50,2018,1,"Cleveland, OH",41.505493,-81.681290,18122.0,2251858.0,124.26,51.0,4,129.27,96.12
51,2018,1,"Chattanooga, TN-GA",35.045631,-85.309677,7766.0,925329.0,119.15,52.0,4,129.27,92.17


In [82]:
jan_tier4.to_csv('jan_tier4.csv', index = False)

#### Concatenate the 4 dataframes consisting of each Tier

In [84]:
jan_18_ranked = pd.concat([jan_tier1, jan_tier2, jan_tier3, jan_tier4])
jan_18_ranked.head(29)

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR,Comp_Set_Index
0,2018,1,"San Francisco/San Mateo, CA",37.733795,-122.446747,196266.0,79262664.0,403.85,1.0,1,269.33,149.95
1,2018,1,Colorado Area,39.482231,-106.046181,62477.0,22677905.0,362.98,2.0,1,269.33,134.77
2,2018,1,Florida Keys,24.778338,-80.921608,14134.0,4454393.0,315.15,3.0,1,269.33,117.01
3,2018,1,"Oahu Island, HI",21.483521,-158.245433,36974.0,11292040.0,305.40,4.0,1,269.33,113.39
4,2018,1,"Miami/Hialeah, FL",25.761681,-80.191788,176115.0,52805582.0,299.84,5.0,1,269.33,111.33
5,2018,1,"West Palm Beach/Boca Raton, FL",26.709723,-80.064163,64499.0,17679178.0,274.10,6.0,1,269.33,101.77
6,2018,1,"Fort Lauderdale, FL",26.139412,-80.133591,75711.0,20507722.0,270.87,7.0,1,269.33,100.57
7,2018,1,"New York, NY",40.730610,-73.935242,163800.0,41590578.0,253.91,8.0,1,269.33,94.27
8,2018,1,Florida Central,28.007698,-81.839268,77699.0,19381520.0,249.44,9.0,1,269.33,92.62
9,2018,1,"Phoenix, AZ",33.448376,-112.074036,227198.0,54597598.0,240.31,10.0,1,269.33,89.23


In [85]:
jan_18_ranked.tail()

,Year,Month,Region,Lat,Lng,GroupDemandTY,GroupRevenueTY,MonthADR,Rank,Tier,Tier_ADR,Comp_Set_Index
51,2018,1,"Chattanooga, TN-GA",35.045631,-85.309677,7766.0,925329.0,119.15,52.0,4,129.27,92.17
52,2018,1,"Norfolk/Virginia Beach, VA",36.850769,-76.285873,19217.0,2157129.0,112.25,53.0,4,129.27,86.83
53,2018,1,"St Louis, MO-IL",38.627003,-90.199402,53512.0,5942624.0,111.05,54.0,4,129.27,85.91
54,2018,1,"Knoxville, TN",35.964668,-83.926453,12926.0,1323684.0,102.40,55.0,4,129.27,79.21
55,2018,1,"Myrtle Beach, SC",33.689060,-78.886696,28000.0,2354348.0,84.08,56.0,4,129.27,65.04


In [86]:
jan_18_ranked.to_csv('jan_18_ranked.csv', index = False)